# Taller 3. Filtrado de marcadores moleculares tipo SNP
Genética de rasgos complejos 2024-01 - Docente Johana Carolina Soto Sedano

__Hans D. Escobar H.__ ([hdescobarh](https://github.com/hdescobarh))

Como en la guía, para familiarizarme los datos aplicare los pasos de forma interactiva, pero con R + Jupyter Notebook en lugar de Excel.

Filtros a aplicar:
1. Marcadores monomórficos y multialélicos
2. Marcadores con segregación idéntica
3. Marcadores con MAF < 5%

## Conociendo los datos


In [1]:
data_file <- "Private_Data/GBS.txt"
data <- read.delim(data_file, row.names = 1, colClasses = "character")
data <- subset(data, select = -c(3:10))
n_marcadores_iniciales <- nrow(data)
n_genotipos_iniciales <- ncol(data) - 2
cat(
  "Número marcadores:", n_marcadores_iniciales,
  "\nNúmero de genotipos:", n_genotipos_iniciales
)
head(data)


Número marcadores: 83830 
Número de genotipos: 150

,alleles,chrom,G1,G2,G3,G4,G5,G6,G7,G8,⋯,G141,G142,G143,G144,G145,G146,G147,G148,G149,G150
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
SNP1,C/T,ST4.03ch01,CC,CC,CC,CC,CC,CC,CC,CC,⋯,CC,CC,CC,CC,CC,CC,CC,CC,CC,CC
SNP2,A/G,ST4.03ch01,AG,AG,AA,AA,AG,AA,AA,AA,⋯,AG,AA,AA,AA,AA,AA,AA,AA,AA,AA
SNP3,C/A,ST4.03ch01,CC,CC,CC,CC,CC,CC,CC,CC,⋯,CC,CC,CC,CC,CC,CC,CC,CC,CC,CC
SNP4,G/A,ST4.03ch01,GA,GA,GG,GG,GA,GG,GG,GG,⋯,GA,GG,GG,GG,GG,GG,GG,GG,GG,GG
SNP5,C/T,ST4.03ch01,CT,CT,TT,CT,CT,TT,TT,TT,⋯,CT,TT,TT,TT,TT,TT,TT,TT,CT,TT
SNP6,G/A,ST4.03ch01,GG,GG,GG,GG,GG,GG,GG,GG,⋯,GG,GG,GG,GG,GG,GG,GG,GG,GA,GG


In [2]:
imprimir_estado <- function(
    data, n_marcadores_iniciales, n_genotipos_iniciales) {
  cat(sprintf(
    "Número marcadores: %i/%i\nNúmero de genotipos: %i/%i",
    nrow(data), n_marcadores_iniciales, ncol(data) - 2, n_genotipos_iniciales
  ))
}


## Filtro Marcadores monomórficos y multialélicos

In [3]:
data <- subset(
  data, grepl("^[AGCT]/[AGCT]$", data[, "alleles"], ignore.case = TRUE),
)
imprimir_estado(data, n_marcadores_iniciales, n_genotipos_iniciales)


Número marcadores: 83830/83830
Número de genotipos: 150/150

## Filtro marcadores con segregación idéntica

In [4]:
# Se puede guardar los duplicados en caso de que se quieran explorar en detalle
marcadores_duplicados <- duplicated(data[, 3:ncol(data)])
data <- data[!marcadores_duplicados, ]
imprimir_estado(data, n_marcadores_iniciales, n_genotipos_iniciales)
head(data)


Número marcadores: 68189/83830
Número de genotipos: 150/150

,alleles,chrom,G1,G2,G3,G4,G5,G6,G7,G8,⋯,G141,G142,G143,G144,G145,G146,G147,G148,G149,G150
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
SNP1,C/T,ST4.03ch01,CC,CC,CC,CC,CC,CC,CC,CC,⋯,CC,CC,CC,CC,CC,CC,CC,CC,CC,CC
SNP2,A/G,ST4.03ch01,AG,AG,AA,AA,AG,AA,AA,AA,⋯,AG,AA,AA,AA,AA,AA,AA,AA,AA,AA
SNP3,C/A,ST4.03ch01,CC,CC,CC,CC,CC,CC,CC,CC,⋯,CC,CC,CC,CC,CC,CC,CC,CC,CC,CC
SNP4,G/A,ST4.03ch01,GA,GA,GG,GG,GA,GG,GG,GG,⋯,GA,GG,GG,GG,GG,GG,GG,GG,GG,GG
SNP5,C/T,ST4.03ch01,CT,CT,TT,CT,CT,TT,TT,TT,⋯,CT,TT,TT,TT,TT,TT,TT,TT,CT,TT
SNP6,G/A,ST4.03ch01,GG,GG,GG,GG,GG,GG,GG,GG,⋯,GG,GG,GG,GG,GG,GG,GG,GG,GA,GG


## Filtro marcadores con MAF < 5%

In [5]:
# Calcula el alelo menos frecuente a partir de los genotipos llamados y
# retorna TRUE sí pasa el filtro
filtro_maf <- function(fila, umbral = 0.05) {
  conteo_alelos <- c(0, 0)
  names(conteo_alelos) <- unlist(strsplit(fila["alleles"], split = "/"))
  total <- 0
  for (genotipo in fila[3:length(fila)]) {
    if (grepl("NN", genotipo, ignore.case = TRUE)) {
      next
    }
    total <- total + 2
    for (alelo in unlist(strsplit(genotipo, split = ""))) {
      conteo_alelos[alelo] <- conteo_alelos[alelo] + 1
    }
  }
  (min(conteo_alelos) / total) >= umbral
}


In [6]:
data <- data[apply(data, 1, filtro_maf), ]
imprimir_estado(data, n_marcadores_iniciales, n_genotipos_iniciales)
data


Número marcadores: 45849/83830
Número de genotipos: 150/150

,alleles,chrom,G1,G2,G3,G4,G5,G6,G7,G8,⋯,G141,G142,G143,G144,G145,G146,G147,G148,G149,G150
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
SNP2,A/G,ST4.03ch01,AG,AG,AA,AA,AG,AA,AA,AA,⋯,AG,AA,AA,AA,AA,AA,AA,AA,AA,AA
SNP4,G/A,ST4.03ch01,GA,GA,GG,GG,GA,GG,GG,GG,⋯,GA,GG,GG,GG,GG,GG,GG,GG,GG,GG
SNP5,C/T,ST4.03ch01,CT,CT,TT,CT,CT,TT,TT,TT,⋯,CT,TT,TT,TT,TT,TT,TT,TT,CT,TT
SNP6,G/A,ST4.03ch01,GG,GG,GG,GG,GG,GG,GG,GG,⋯,GG,GG,GG,GG,GG,GG,GG,GG,GA,GG
SNP7,A/G,ST4.03ch01,AA,NN,AG,AA,AG,AA,AA,AA,⋯,AA,AA,AA,AA,AA,AA,AA,AA,AA,AA
SNP17,G/A,ST4.03ch01,GA,GA,AA,GA,GA,AA,AA,AA,⋯,AA,GA,NN,AA,AA,AA,AA,AA,GA,GA
SNP22,G/A,ST4.03ch01,GA,GA,GA,GA,GA,GG,GA,GG,⋯,GA,GA,GG,GA,NN,GA,GA,GA,GA,GA
SNP25,C/T,ST4.03ch01,CC,CC,CT,CT,CC,CC,CC,CC,⋯,CT,CC,CC,CC,CC,CC,CC,CC,CC,CC
SNP28,G/A,ST4.03ch01,NN,NN,GG,GG,NN,GA,GA,GA,⋯,GG,GG,GA,GA,GG,GA,GA,GA,GG,GA
